# Making Loan Granting Decisions

In this project, we have a loan dataset from a bank. The data contain all the loans asked to the bank, whether the bank decided to grant it, whether the borrower managed to repay it, and the information about the borrower at the moment he/she is asking for the loan. The goal is to build a model to predict whether a customer will repay, which can be used to decide whether to grant loans to future customers. **This project is stll a work in progress.**

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score,confusion_matrix,classification_report
from sklearn.metrics import precision_score,recall_score,f1_score,make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df_loan  = pd.read_csv('data/loan_table.csv')
df_borrower  = pd.read_csv('data/borrower_table.csv')

# inspect the size of the data
df_loan.shape, df_borrower.shape

((101100, 5), (101100, 12))

In [3]:
df_loan.head()

,loan_id,loan_purpose,date,loan_granted,loan_repaid
0,19454,investment,2012-03-15,0,NaN
1,496811,investment,2012-01-17,0,NaN
2,929493,other,2012-02-09,0,NaN
3,580653,other,2012-06-27,1,1.0
4,172419,business,2012-05-21,1,0.0


In [4]:
df_borrower.head()

,loan_id,is_first_loan,fully_repaid_previous_loans,currently_repaying_other_loans,total_credit_card_limit,avg_percentage_credit_card_limit_used_last_year,saving_amount,checking_amount,is_employed,yearly_salary,age,dependent_number
0,289774,1,NaN,NaN,8000,0.49,3285,1073,0,0,47,3
1,482590,0,1.0,0.0,4500,1.03,636,5299,1,13500,33,1
2,135565,1,NaN,NaN,6900,0.82,2085,3422,1,24500,38,8
3,207797,0,1.0,0.0,1200,0.82,358,3388,0,0,24,1
4,828078,0,0.0,0.0,6900,0.80,2138,4282,1,18100,36,1


## Data Cleaning

In [5]:
# Merge the two data frames using "loan_id"
df = pd.merge(left=df_loan, right=df_borrower, how='left', on='loan_id')
# Drop "loan_id" and "date", which are not useful for modeling
df = df.drop(['loan_id','date'],axis=1)

# Rename some columns whose names are too long 
df = df.rename(columns={'fully_repaid_previous_loans': 'previous_repaid', 
                        'currently_repaying_other_loans': 'repaying',
                        'total_credit_card_limit': 'credit_limit', 
                        'avg_percentage_credit_card_limit_used_last_year':'limit_used'})

In [6]:
# Check the amount of NANs in each column
df.isnull().sum()

loan_purpose            0
loan_granted            0
loan_repaid         53446
is_first_loan           0
previous_repaid     54947
repaying            54947
credit_limit            0
limit_used           6972
saving_amount           0
checking_amount         0
is_employed             0
yearly_salary           0
age                     0
dependent_number        0
dtype: int64

There are 53446 NANs in ``loan_repaid``, which should be due to the fact that those customers were not granted the loan in the first place. We can check whether the number of granted loans is also 53446.

In [7]:
# Yes, it is also 53446
print sum(df.loan_granted == 0)

53446


There are 54947 NANs in both ``previous_repaid`` and ``repaying``, which should be due to the fact that those customers never had a loan in the past. We can check whether there are 54947 ones in ``is_first_loan`` 

In [8]:
# Yes, it is also 54947
print sum(df.is_first_loan)

54947


In [9]:
# fill the NANs in "previous_repaid" and "repaying" and make both features 
# have three categories, "NA", "YES", "NO"

df.previous_repaid.fillna(-1,inplace=True)
df.repaying.fillna(-1,inplace=True)
df.previous_repaid = df.previous_repaid.replace({0: 'No', 1: 'Yes', -1: 'NA'})
df.repaying = df.repaying.replace({0: 'No', 1: 'Yes', -1: 'NA'})

Now we don't need ``is_first_loan`` anymore, because this information is in ``previous_repaid`` and ``repaying`` already.

In [10]:
df = df.drop(['is_first_loan'],axis=1)

Lastly, there are 6972 NANs in ``limit_used``, which should be due to the fact that those customers have no credit card limit at all. We can check whether there are 6972 zeros in ``credit_limit`` 

In [11]:
# Yes, it is also 6972
print sum(df.credit_limit == 0) 

6972


In [12]:
# We can just fill these NANs using zeros
df.limit_used.fillna(0,inplace=True)

The data have the total saving account balance and total checking account balance separately. However, what matters is the total balance, as some people prefer to put most of their money in the checking accounts while some others do not. So here I create a new column ``savings``, which equals the total balance. And then ``saving_amount`` and ``checking_amount`` can be dropped.

In [13]:
print df[['saving_amount','checking_amount']].head(10)

df['savings'] =  df.saving_amount + df.checking_amount
df = df.drop(['saving_amount','checking_amount'],axis=1)

   saving_amount  checking_amount
0           1491             6285
1            141             5793
2            660             3232
3           3345             2764
4           1050             3695
5           1028             3269
6           4206             4368
7            886             1597
8           1837             3795
9            134             1408


## Exploratory Data Analysis and Visualization

**To be completed...**


## Data Transformation and Train/Test Split

First, we have to transform the categories in categorical features to numerical values.

In [14]:
# Use dummy variables
df = pd.get_dummies(df,columns=['loan_purpose','previous_repaid','repaying'])
# Drop one dummy variable for each of the three features to avoid dummy variable trap
df = df.drop(['loan_purpose_other','previous_repaid_NA','repaying_NA'],axis=1)

Now we have to split the data into three. First, the customers who were not granted loans have to be taken out because we don't know whetehr they would have repaid. Thus, they cannot be used for modeling. Secondly, I will split the remaining data into a training set and a test set, using a 70/30 split

In [15]:
# Those that were not granted loans
X_notgranted = df[df.loan_granted == 0].reset_index(drop=True).drop(['loan_granted','loan_repaid'],axis=1)

# Those that were granted loans
X = df[df.loan_granted == 1].reset_index(drop=True).drop(['loan_granted','loan_repaid'],axis=1)
Y = df[df.loan_granted == 1].reset_index(drop=True)['loan_repaid'].values # target variable

# Train/Test split
X_train, X_test, Y_train, Y_test =  train_test_split(X, Y, test_size=0.3, random_state=0)

Because not all the features have the same scale, here I scale them using ``StandardScaler``. The standardizer is fit to the training set only, and then used to transform both the training and test sets as well as the data for the costumers that were not granted loans.

In [16]:
scaler = StandardScaler()

# After the performing the standardization, transform them back to data frames
# in order to keep the column names
X_train = pd.DataFrame(scaler.fit_transform(X_train.values), columns=X.columns)
X_test = pd.DataFrame(scaler.transform(X_test.values), columns=X.columns)
X_notgranted = pd.DataFrame(scaler.transform(X_notgranted.values), columns=X.columns)

## Random Forest Classifier

Here I try a Random Forest classifier and use ``GridSearchCV`` to tune the hyperparameters. In this loan granting problem, both precision (the fraction of people who eventually repay among all the borrowers) and recall (the fraction of costumers who are granted loans among all of those who would repay) are important. I therefore use F1 score as the main evaluation metric.

In [17]:
rf = RandomForestClassifier()

# Use F1 score for GridSearchCV
f1 = make_scorer(f1_score)

# Specify parameters and distributions to sample from
parameters = {"n_estimators": [100],
              "max_features": [2,3,4,5],
              "min_samples_split": [2,3,4,5],
              "min_samples_leaf": [1,2,3,4,5],
              "criterion": ["entropy"],
              "random_state": [0]}

# Run Grid Search
grid_search = GridSearchCV(estimator = rf,
                           param_grid = parameters,
                           scoring = f1,
                           n_jobs = -1)

grid_search = grid_search.fit(X_train, Y_train)

print grid_search.best_score_
print grid_search.best_params_
rf = grid_search.best_estimator_

0.924426650427
{'min_samples_leaf': 3, 'n_estimators': 100, 'min_samples_split': 2, 'random_state': 0, 'criterion': 'entropy', 'max_features': 4}


In [18]:
Y_pred = rf.predict(X_test)
Y_train_pred = rf.predict(X_train)

print "Accuracy:"
print "Training Set:",rf.score(X_train, Y_train)
print "Test Set:",rf.score(X_test, Y_test)

tn, fp, fn, tp = confusion_matrix(Y_test, Y_pred).ravel()
print "\nConfusion Matrix:"
print "True Positive:", tp
print "False Negative:", fn
print "True Negative:", tn
print "False Positive:", fp

print "\nPrecision:"
print "Training Set:",precision_score(Y_train, Y_train_pred)
print "Test Set:",precision_score(Y_test, Y_pred)

print "\nRecall:"
print "Training Set:",recall_score(Y_train, Y_train_pred)
print "Test Set:",recall_score(Y_test, Y_pred)

print "\nF1 Score:"
print "Training Set:",f1_score(Y_train, Y_train_pred)
print "Test Set:",f1_score(Y_test, Y_pred)

Accuracy:
Training Set: 0.966094073208
Test Set: 0.902217248374

Confusion Matrix:
True Positive: 8493
False Negative: 793
True Negative: 4406
False Positive: 605

Precision:
Training Set: 0.979849581382
Test Set: 0.933501868543

Recall:
Training Set: 0.967086834734
Test Set: 0.914602627611

F1 Score:
Training Set: 0.97342637626
Test Set: 0.923955613577


In [19]:
Y_notgranted_pred = rf.predict(X_notgranted)

# The fraction of people who would have been granted loans based
# on my model among those costumers who were not granted loans
print Y_notgranted_pred.sum()/len(Y_notgranted_pred)

0.344291434345


**To be continue...**